In [ ]:
project = 'saga-nvdb-prod-vlmh'
use_colab_auth = True

# Legg inn ditt eget prosjekt her, f.eks. 'saga-olanor-playground-ab12'
bq_job_project = ''

In [ ]:
if (use_colab_auth):
  from google.colab import auth
  auth.authenticate_user()
  print('Authenticated')

In [ ]:
import warnings
from google.cloud import bigquery

warnings.filterwarnings('ignore')
client = bigquery.Client(project=bq_job_project)

Denne spørringen henter tunnelnavn, åpningsår og antall tunnelløp per tunnel. Den viser også vegsystemreferansens kortform for alle tunnelløpene hver tunnel har.

In [ ]:
query = f"""
SELECT
  tunnel.egenskaper.navn as tunnelnavn,
  tunnel.egenskaper.aapningsaar,
  COUNT(tunnellop) antall_lop,
  ARRAY_AGG(tunnellop_vegsystemreferanser.kortform) tunnellop_vegsystemreferanse_kortform,
FROM
  `{project}.curated.tunneler_med_lop` tunnel,
  UNNEST(tunnellop) tunnellop,
  UNNEST(tunnellop.lokasjon.vegsystemreferanser) tunnellop_vegsystemreferanser
WHERE
  tunnel.metadata.sluttdato IS NULL
GROUP BY tunnel.egenskaper.navn, tunnel.egenskaper.aapningsaar
ORDER BY aapningsaar DESC, antall_lop DESC
"""

print(query)

client.query(query).to_dataframe()